In [ ]:
# importando as bibliotecas necessárias
from pyspark.sql import SparkSession

import requests
import datetime

# recuperando a Spark Session
spark = SparkSession.builder.appName("mr-data-test").getOrCreate()

# recuperando o Spark Context
sc = spark.sparkContext

In [ ]:
# Função para consulta do índice do CDI
def get_fator(dates):
    url = "https://calculadorarendafixa.com.br/calculadora/di/calculo"

    response = requests.get(
        url,
        params=[
            ('dataInicio', dates[0].strftime("%Y-%m-%d")),
            ('dataFim', dates[1].strftime("%Y-%m-%d")),
            ('percentual', '100'),
            ('valor', '1000.00'),
        ]
    ).json()

    return {
        "Data": dates[1],
        "Fator": response["fator"]
    }

In [ ]:
# testando
response = get_fator((datetime.datetime(2021, 1, 5), datetime.datetime(2021, 1, 6)))
print(response)

{'Data': datetime.datetime(2021, 1, 6, 0, 0), 'Fator': '1.00007469'}

In [ ]:
# definição do período a ser consultado
start = datetime.datetime(2021, 7, 20)
end = datetime.datetime(2021, 7, 26)

date_list = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

In [ ]:
# gerando um rdd a partir do período
rdd = sc.parallelize(list(zip(date_list, date_list[1:])))

In [ ]:
# exibindo informações do rdd
print("Partitions: "+str(rdd.getNumPartitions()))
print("1st element: "+str(rdd.first()))
print(rdd)
print("Number of elements in RDD -> {counts} ".format(counts=rdd.count()))

Partitions: 8
1st element: (datetime.datetime(2021, 7, 20, 0, 0), datetime.datetime(2021, 7, 21, 0, 0))
ParallelCollectionRDD[107] at readRDDFromInputStream at PythonRDD.scala:413
Number of elements in RDD -> 5

In [ ]:
# recupera o fator do CDI para cada data no rdd
rdd = rdd.map(get_fator)

In [ ]:
# exibindo o primeiro registro
rdd.first()

Out[8]: {'Data': datetime.datetime(2021, 7, 21, 0, 0), 'Fator': '1.00016137'}

In [ ]:
# cria um dataframe a partir do rdd
df = spark.createDataFrame(rdd)

In [ ]:
# verificando por informações faltantes
from pyspark.sql import functions as F
df.where(F.isnull(F.col("Fator"))).count()

Out[10]: 0

In [ ]:
# exibindo o schema do df
df.printSchema()

root
-- Data: timestamp (nullable = true)
-- Fator: string (nullable = true)

In [ ]:
# transformando coluna fator para double
df = df.withColumn("Fator",df["Fator"].cast('double'))

# transformando data fator para date
df = df.withColumn("Data",df["Data"].cast('date'))

df.printSchema()

root
-- Data: date (nullable = true)
-- Fator: double (nullable = true)

In [ ]:
# calculando Fator Acumulado
from pyspark.sql import functions as F, Window, types
from functools import reduce
from operator import mul

window = Window.orderBy('Data')

mul_udf = F.udf(lambda x: reduce(mul, x), types.DoubleType())

df = df.withColumn('FatorAcumulado', mul_udf(F.collect_list(F.col('Fator')).over(window)))

In [ ]:
# calculando Taxa Acumulada
df = df.withColumn('TaxaAcumulada', (100 * (df["FatorAcumulado"] - 1)))

In [ ]:
# exibindo os 2 primeiros registros
df.show(n=2)

+----------+----------+-----------------+--------------------+
 Data| Fator| FatorAcumulado| TaxaAcumulada|
+----------+----------+-----------------+--------------------+
2021-07-21|1.00016137| 1.00016137|0.016137000000004953|
2021-07-22|1.00016137|1.000322766040277| 0.03227660402769228|
+----------+----------+-----------------+--------------------+
only showing top 2 rows

In [ ]:
# gravando dados em uma tabela delta
# %fs rm -r /tmp/delta-table/mr-data-test
df.write.format("delta").mode("overwrite").save("/tmp/delta-table/mr-data-test")

In [ ]:
# lendo os dados da tabela delta
df = spark.read.format("delta").load("/tmp/delta-table/mr-data-test")
df.show()

+----------+----------+------------------+--------------------+
 Data| Fator| FatorAcumulado| TaxaAcumulada|
+----------+----------+------------------+--------------------+
2021-07-21|1.00016137| 1.00016137|0.016137000000004953|
2021-07-22|1.00016137| 1.000322766040277| 0.03227660402769228|
2021-07-23|1.00016137|1.0004841881250328|0.048418812503281394|
2021-07-24|1.00016137|1.0006456362584706| 0.06456362584705833|
2021-07-25| 1.0|1.0006456362584706| 0.06456362584705833|
+----------+----------+------------------+--------------------+